scratch environment
===================

In [ ]:
// load data

import org.lamastex.spark.trendcalculus._
import spark.implicits._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import java.sql.Timestamp
import org.apache.spark.sql.expressions._



  

>     import org.lamastex.spark.trendcalculus._
>     import spark.implicits._
>     import org.apache.spark.sql._
>     import org.apache.spark.sql.functions._
>     import java.sql.Timestamp
>     import org.apache.spark.sql.expressions._

In [ ]:

val oilDS = spark.read.fx1m("dbfs:/FileStore/shared_uploads/fabiansi@kth.se/*csv.gz").toDF.withColumn("ticker", lit("BCOUSD")).select($"ticker", $"time" as "x", $"close" as "y").as[TickerPoint].orderBy("time")

// store loaded data as temp view
// oilDS.createOrReplaceTempView("temp")

  

>     oilDS: org.apache.spark.sql.Dataset[org.lamastex.spark.trendcalculus.TickerPoint] = [ticker: string, x: timestamp ... 1 more field]

In [ ]:
//add additional usefull columns to the dataframe

import org.apache.spark.sql.expressions._

val windowSpec = Window.orderBy("x")

val oilDS2 = oilDS
.withColumn("diff_close", $"y" - when((lag("y", 1).over(windowSpec)).isNull, 0).otherwise(lag("y", 1).over(windowSpec)))
.withColumn("index", row_number().over(windowSpec))


// store loaded data as temp view
oilDS2.createOrReplaceTempView("temp")

  

>     import org.apache.spark.sql.expressions._
>     windowSpec: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@69dc2e23
>     oilDS2: org.apache.spark.sql.DataFrame = [ticker: string, x: timestamp ... 3 more fields]

In [ ]:
'''
cell is used for testing pyspark syntax only
'''

import datetime
import numpy as np


oilDS_py = spark.table("temp") # data: Row(ticker=u'BCOUSD', x=datetime.datetime(2010, 11, 14, 20, 21), y=86.74), ...
# print(oilDS_py.take(10))
start = datetime.datetime(2010, 11, 14, 20, 19)
end = datetime.datetime(2010, 11, 14, 20, 32)

# oilDS_py.filter(oilDS_py.index.between(3, 6))

# rdd = oilDS_py.filter(oilDS_py["x"] > start).filter(oilDS_py["x"] <= end).select('diff_close').rdd.map(list)

# print(rdd)

# np.array(oilDS_py.where(oilDS_py.index == 5).select('diff_close').collect()).item()

In [ ]:
import gym
import math
import numpy as np

PENALTY = 1  # 0.999756079

class MarketEnv(gym.Env):
  def __init__(self, full_data, start_date, end_date, scope = 60, sudden_death = -1., cumulative_reward = False):
    
    self.cumulative_reward = cumulative_reward
    self.actions = ["LONG", "SHORT"] # pass not required, if already owned, buy is hold
    self.action_space = gym.spaces.Discrete(len(self.actions))
    self.state = None
    
    # self.time_range = full_data.filter(full_data["x"] > start_date).filter(full_data["x"] <= end_date)
    self.time_range = np.array(full_data.filter(full_data["x"] > start_date).filter(full_data["x"] <= end_date).select('diff_close').collect())

    print('time_range.shape: {}'.format(self.time_range.shape))
    
    self.sudden_death = sudden_death
    self.scope = scope
    self.time_index = 0
    self.reset()
    
    
  def step(self, action):
    if self.done:
      return self.state, self.reward, self.done, {}
    
    self.reward = 0
  
    if self.actions[action] == "LONG":
      if sum(self.boughts) < 0:
        for b in self.boughts:
            self.reward += -(b + 1)
        if self.cumulative_reward:
            self.reward = self.reward / max(1, len(self.boughts))
        if self.sudden_death * len(self.boughts) > self.reward:
            self.done = True
        self.boughts = []
      self.boughts.append(1.0)
      
    elif self.actions[action] == "SHORT":
      if sum(self.boughts) > 0:
        for b in self.boughts:
          self.reward += b - 1
        if self.cumulative_reward:
          self.reward = self.reward / max(1, len(self.boughts))
        if self.sudden_death * len(self.boughts) > self.reward:
          self.done = True
        self.boughts = []
      self.boughts.append(-1.0)
      
    else:
      raise ValueError

    # vari = np.array(self.time_range.where(self.time_range.index == self.time_index).select('diff_close').collect()).item()
    vari = self.time_range[self.time_index]
    
    self.cum = self.cum * (1 + vari)

    for i in range(len(self.boughts)):
      self.boughts[i] = self.boughts[i] * PENALTY * (1 + vari * (-1 if sum(self.boughts) < 0 else 1))

    self.defineState()
    self.time_index += 1
    # if self.time_index > self.time_range.count() - self.scope:
    if self.time_index > self.time_range.shape[0] - self.scope:
      self.done = True

    if self.done:
      for b in self.boughts:
        self.reward += (b * (1 if sum(self.boughts) > 0 else -1)) - 1
      if self.cumulative_reward:
        self.reward = self.reward / max(1, len(self.boughts))

      self.boughts = []

    return self.state, self.reward, self.done  
  
  
  def reset(self, return_state=False):
    self.boughts = []
    self.cum = 1.
    self.done = False
    self.reward = 0 
    self.time_index = self.scope
    self.defineState()
  
    if return_state:
      return self.state
  
  def defineState(self):
    tmpState = []

    budget = (sum(self.boughts) / len(self.boughts)) if len(self.boughts) > 0 else 1.
    size = math.log(max(1., len(self.boughts)), 100)
    position = 1. if sum(self.boughts) > 0 else 0.
    tmpState.append([[budget, size, position]])

    # need to change here!! 
    # df_back = self.time_range.filter(self.time_range.index.between(self.time_index, self.time_index + self.scope - 1))
    np_back = self.time_range[self.time_index:self.time_index + self.scope]
    # TODO check if we go out of range
    
    # np_back = np.array(df_back.select('diff_close').collect())
    # print('[!!] {}'.format(np_back))
    
    tmpState.append(np_back)

    tmpState = [np.array(i) for i in tmpState]
    self.state = tmpState
  
  
  def seed(self):
    pass
    
    

In [ ]:
start = datetime.datetime(2010, 11, 14, 20, 19)
end = datetime.datetime(2012, 11, 14, 20, 32)

env = MarketEnv(oilDS_py, start, end)

print(env.step(0))



In [ ]:
print(env.step(0))

  

reinforcement learning part
===========================

-   see
    https://dbc-635ca498-e5f1.cloud.databricks.com/?o=445287446643905\#notebook/4201196137758409/command/4201196137758420

In [ ]:
class ExperienceReplay:
  def __init__(self, max_memory=100, discount=.9):
    self.max_memory = max_memory
    self.memory = list()
    self.discount = discount

  def remember(self, states, game_over):
    # memory[i] = [[state_t, action_t, reward_t, state_t+1], game_over?]
    self.memory.append([states, game_over])
    if len(self.memory) > self.max_memory:
      del self.memory[0]

  def get_batch(self, model, batch_size=10):
    len_memory = len(self.memory)
    num_actions = model.output_shape[-1]
    # print(self.memory[0][0][0])
    
    env_dim = self.memory[0][0][0].shape[1]
    inputs = np.zeros((min(len_memory, batch_size), env_dim))
    targets = np.zeros((inputs.shape[0], num_actions))
    for i, idx in enumerate(np.random.randint(0, len_memory, size=inputs.shape[0])):
      state_t, action_t, reward_t, state_tp1 = self.memory[idx][0]
      game_over = self.memory[idx][1]

      inputs[i:i+1] = state_t
      # There should be no target values for actions not taken.
      # Thou shalt not correct actions not taken #deep
      targets[i] = model.predict(state_t)[0]
      Q_sa = np.max(model.predict(state_tp1)[0])
      if game_over:  # if game_over is True
        targets[i, action_t] = reward_t
      else:
        # reward_t + gamma * max_a' Q(s', a')
        targets[i, action_t] = reward_t + self.discount * Q_sa
    return inputs, targets

In [ ]:
import json
import numpy as np

from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras import losses
from keras import backend as k
import tensorflow as tf

from elephas.utils.rdd_utils import to_simple_rdd
from elephas.spark_model import SparkModel

import collections
import datetime

epsilon = .1  # exploration
num_actions = 2  # [long, short]
epoch = 400
max_memory = 500
hidden_size = 100
batch_size = 50
sequence_scope = 60

model = Sequential()
model.add(Dense(hidden_size, input_shape=(sequence_scope,), activation='relu'))
model.add(Dense(hidden_size, activation='relu'))
model.add(Dense(num_actions))
model.compile(loss='mse', optimizer='adam')



# Define environment

start = datetime.datetime(2010, 11, 14, 20, 19)
end = datetime.datetime(2010, 12, 14, 20, 32)

env = MarketEnv(oilDS_py, start, end, scope = sequence_scope, sudden_death = -1., cumulative_reward = False)

# Initialize experience replay object
exp_replay = ExperienceReplay(max_memory=max_memory)

# Train
win_cnt = 0
last_ten = collections.deque(maxlen=10)

for e in range(epoch):
    loss = 0.
    # env.reset()
    game_over = False
    # get initial input
    state = env.reset(return_state=True)
    input_t = state[1].reshape(1, sequence_scope)
    
    counter = 0
    
    while not game_over:
        # printing stuff
        # print('not over, count: {}'.format(counter))
        counter += 1
        
        input_tm1 = input_t
        # get next action
        if np.random.rand() <= epsilon:
            action = np.random.randint(0, num_actions, size=1).item()
        else:
            q = model.predict(input_tm1)
            action = np.argmax(q[0])

        # if not isinstance(action, int):
        #   action = action[0]
          
        print('action: {} action type: {}'.format(action, type(action)))
            
        # apply action, get rewards and new state
        state, reward, game_over = env.step(action)
        input_t = state[1].reshape(1, sequence_scope)
        
        if reward == 1:
            win_cnt += 1            

        # store experience
        exp_replay.remember([input_tm1, action, reward, input_t], game_over)

        # adapt model
        inputs, targets = exp_replay.get_batch(model, batch_size=batch_size)

        # print("[!] before grad computation")
        # mseloss = losses.mean_squared_error(targets, model.output)
        # gradients = k.gradients(mseloss, model.trainable_weights)
        # evaluated_gradients = sess.run(gradients, feed_dict={model.input: inputs})
        # print("[!] eval grads: {}".format(evaluated_gradients)
        
        train_rdd = to_simple_rdd(sc, inputs, targets)
        
        spark_model = SparkModel(model, frequency='epoch', mode='asynchronous')
        print("[!] before model fit")
        spark_model.fit(train_rdd, epochs=20, batch_size=32, verbose=0, validation_split=0.1)
        model = spark_model._master_network # hack!
        
        # loss += model.train_on_batch(inputs, targets)
        
    last_ten.append((reward+1)/2)
    print("Epoch {:03d}/{:d} | Loss {:.4f} | Win count {} | Last 10 win rate {}".format(e, epoch - 1, loss, win_cnt, sum(last_ten)/10.0))

  

>     time_range.shape: (25484, 1)
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 1 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 1 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 1 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 1 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 1 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 1 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 1 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 1 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 1 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>
>     *** WARNING: skipped 7599 bytes of output ***
>
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 1 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 1 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 1 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 1 action type: <class 'int'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load
>     >>> Async training complete.
>     action: 0 action type: <class 'numpy.int64'>
>     [!] before model fit
>     >>> Fit model
>     >>> Initialize workers
>     >>> Distribute load

  

elephas test
============

In [ ]:
rdd = oilDS_py.filter(oilDS_py["x"] > start).filter(oilDS_py["x"] <= end).select('diff_close').rdd.map(list)
print(rdd)





  

read data as structures stream
==============================

In [ ]:
val oil_path = "dbfs:/FileStore/shared_uploads/fabiansi@kth.se/*csv.gz"

val input = spark
  .readStream
  .format("delta")
  .load(oil_path)
  .as[TickerPoint]


In [ ]:
joinedDS = spark.read.parquet("dbfs:/FileStore/shared_uploads/fabiansi@kth.se/joinedDSWithMaxRev").orderBy("x")
